# Linear programming - A Transportation Problem
## The Beer Distribution Problem.
Follows the [Transportation Problem](https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html) example from the Pulp documentation.

### Problem Description
A brewery has 2 warehouses from which it distributes beer to 5 bars. At the start of every week, each bar sends an order to the brewery for X crates of beer, which is then dispatched from the appropriate warehouse to the bar.    
The brewery would like to have an interactive computer program which they can run week by week to tell them which warehouse should supply which bar so as to minimize the costs of the whole operation.    
For example, suppose that at the start of a given week the brewery has 1000 cases at warehouse A, and 4000 cases at warehouse B, and that the bars require 500, 900, 1800, 200, and 700 cases respectively. Which warehouse should supply which bar?

For transportation problems, using a graphical representation of the problem is often helpful during formulation.   
Below is a graphical representation of The Beer Distribution Problem.

<img src="https://coin-or.github.io/pulp/_images/brewery_nodes.jpg" height="300"/>   
<img src="https://coin-or.github.io/pulp/_images/brewery_arcs.jpg" height="300"/>   

### Identify the Decision Variables
In transportation problems we are deciding how to transport goods from their supply nodes to their demand nodes. The decision variables are the Arcs connecting these nodes, as shown in the diagram below. We are deciding how many crates of beer to transport from each warehouse to each pub.

- A1 = number of crates of beer to ship from Warehouse A to Bar 1
- A5 = number of crates of beer to ship from Warehouse A to Bar 5
- B1 = number of crates of beer to ship from Warehouse B to Bar 1
- B5 = number of crates of beer to ship from Warehouse B to Bar 5

Let:

$$W = \{A,B\} \\   
B = \{1, 2, 3, 4, 5 \} \\   
x_{(w,b)} \ge 0 \ldots \forall w \in W, b \in B \\   
x_{(w,b)}  \in \mathbb{Z}^+ \ldots \forall w \in W, b \in B$$

- W: Warehouses
- B: Bars
- $x_{(w,b)} \ge 0$: Number of shipped crates from Warehouse w to Bar b must be greater than or equal to zero for all warehouse and bar combinations. (cannot ship a negative number of crates)
- $x_{(w,b)}  \in \mathbb{Z}^+$ : Number of shipped crates must belong to the set of positive integers \( \mathbb{Z}^+ \) (must ship whole crates, not fraction of a crate)

The lower bound on the variables is Zero, and the values must all be Integers (since the number of crates cannot be negative or fractional). There is no upper bound.

### Formulate the Objective Function
The objective function has been loosely defined as cost. The problem can only be formulated as a linear program if the cost of transportation from warehouse to pub is a linear function of the amounts of crates transported. Note that this is sometimes not the case. This may be due to factors such as economies of scale or fixed costs. For example, transporting 10 crates may not cost 10 times as much as transporting one crate, since it may be the case that one truck can accommodate 10 crates as easily as one. Usually in this situation there are fixed costs in operating a truck which implies that the costs go up in jumps (when an extra truck is required). In these situations, it is possible to model such a cost by using zero-one integer variables: we will look at this later in the course.

We shall assume then that there is a fixed transportation cost per crate. (If the capacity of a truck is small compared with the number of crates that must be delivered then this is a valid assumption). Lets assume we talk with the financial manager, and she gives us the following transportation costs (dollars per crate):

| From Warehouse to Bar  | A | B   |
|-------|-----|---|
| 1     | 2   | 3 | 
| 2     | 4   | 1 | 
| 3     | 5   | 3 | 
| 4     | 2   | 2 | 
| 5     | 1   | 3 | 

#### Objective function
 
**Minimize cost $(z)$**   
$$z = x_{A,1} * cost_{A,1} + x_{A,2} * cost_{A,2} + x_{A,3} * cost_{A,3} + x_{A,4} * cost_{A,4} + x_{A,5} * cost_{A,5} \\   
+ x_{B,1} * cost_{B,1} + x_{B,2} * cost_{B,2} + x_{B,3} * cost_{B,3} + x_{B,4} * cost_{B,4} + x_{B,5} * cost_{B,5}$$   
$$z = \sum_{w=1}^{W} \sum_{b=1}^{B} x_{w,b} * cost_{w,b}$$   

Minimal transportation cost (z) is the sum of nr. of crates (x) multiplied with the corresponding expense (cost) for each route combination (w,b).

### Constraints
The constraints come from considerations of supply and demand. The supply of beer at warehouse A is 1000 cases. The total amount of beer shipped from warehouse A cannot exceed this amount. Similarly, the amount of beer shipped from warehouse B cannot exceed the supply of beer at warehouse B. The sum of the values on all the arcs leading out of a warehouse, must be less than or equal to the supply value at that warehouse:

**Supply constraints from warehouses**
- $ A1 + A2 + A3 + A4 + A5 \leq 1000  <==>   \sum_{b=1}^{B} x_{A,b}  \leq 1000$ $
- $ B1 + B2 + B3 + B4 + B5 \leq 4000  <==>   \sum_{b=1}^{B} x_{B,b}  \leq 4000$ $

The demand for beer at bar 1 is 500 cases, so the amount of beer delivered there must be at least 500 to avoid lost sales. Similarly, considering the amounts delivered to the other bars must be at least equal to the demand at those bars. Note, we are **assuming no penalties for oversupplying bars** (other than the extra transportation cost we incur). We can *balance* the transportation problem to make sure that demand is met exactly - there will be more on this later. For now, the sum of the values on all the arcs leading into a bar, must be greater than or equal to the demand value at that bar:

**Demand constraints from bars**
- $(A1 + B1) >= 500$
- $(A2 + B2) >= 900$
- $(A3 + B3) >= 1800$
- $(A4 + B4) >= 200$
- $(A5 + B5) >= 700$

## 2. Solving linear problem with PuLP

In [1]:
# Import PuLP modeler functions
from pulp import *

In [2]:
# Initialize the model
model = LpProblem("BeerDistribution", LpMinimize) # want to minimize cost
model

BeerDistribution:
MINIMIZE
None
VARIABLES

In [3]:
# Store variables in lists or dictionaries
# Supply nodes
Warehouses = ["A", "B"]
supply = {"A": 1000, "B": 4000}

# Demand nodes
Bars = ["1", "2", "3", "4", "5"]
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}
# Creates a list of costs of each transportation path
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouse
    [3, 1, 3, 2, 3],  # B   Warehouse
]
costs

[[2, 4, 5, 2, 1], [3, 1, 3, 2, 3]]

In [4]:
# The cost data is made into a dictionary
costs = makeDict([Warehouses, Bars], costs, 0)
costs

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'A': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1}),
             'B': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3})})

In [5]:
# quick inspection 
print(type(costs))
print(costs['A']['3'])  # This should output 5

<class 'collections.defaultdict'>
5


In [6]:
# Create list of possible routes
Routes = [(w, b) for w in Warehouses for b in Bars]
Routes

[('A', '1'),
 ('A', '2'),
 ('A', '3'),
 ('A', '4'),
 ('A', '5'),
 ('B', '1'),
 ('B', '2'),
 ('B', '3'),
 ('B', '4'),
 ('B', '5')]

In [7]:
# Initialize decision variables
vars = LpVariable.dicts("Route", (Warehouses, Bars), lowBound=0, upBound=None, cat="Integer")
print(type(vars))
vars

<class 'dict'>


{'A': {'1': Route_A_1,
  '2': Route_A_2,
  '3': Route_A_3,
  '4': Route_A_4,
  '5': Route_A_5},
 'B': {'1': Route_B_1,
  '2': Route_B_2,
  '3': Route_B_3,
  '4': Route_B_4,
  '5': Route_B_5}}

The objective function is added to the variable model using a list comprehension. Since vars and costs are now dictionaries (with further internal dictionaries), they can be used as if they were tables, as for (w,b) in Routes will cycle through all the combinations/arcs. Note that i and j could have been used, but w and b are more meaningful.

In [8]:
# The objective function
model += (lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]), "Sum_of_Transporting_Costs")
model

BeerDistribution:
MINIMIZE
2*Route_A_1 + 4*Route_A_2 + 5*Route_A_3 + 2*Route_A_4 + 1*Route_A_5 + 3*Route_B_1 + 1*Route_B_2 + 3*Route_B_3 + 2*Route_B_4 + 3*Route_B_5 + 0
VARIABLES
0 <= Route_A_1 Integer
0 <= Route_A_2 Integer
0 <= Route_A_3 Integer
0 <= Route_A_4 Integer
0 <= Route_A_5 Integer
0 <= Route_B_1 Integer
0 <= Route_B_2 Integer
0 <= Route_B_3 Integer
0 <= Route_B_4 Integer
0 <= Route_B_5 Integer

The supply and demand constraints are added using a normal for loop and a list comprehension. Supply Constraints: For each warehouse in turn, the values of the decision variables (number of beer cases on arc) to each of the bars is summed, and then constrained to being less than or equal to the supply max for that warehouse. Demand Constraints: For each bar in turn, the values of the decision variables (number on arc) from each of the warehouses is summed, and then constrained to being greater than or equal to the demand minimum.

##### Reminder of the constraints:   
**Supply constraints from warehouses**
- $ A1 + A2 + A3 + A4 + A5 \leq 1000  <==>   \sum_{b=1}^{B} x_{A,b}  \leq 1000$ $
- $ B1 + B2 + B3 + B4 + B5 \leq 4000  <==>   \sum_{b=1}^{B} x_{B,b}  \leq 4000$ $

**Demand constraints from bars**
- $(A1 + B1) >= 500$
- $(A2 + B2) >= 900$
- $(A3 + B3) >= 1800$
- $(A4 + B4) >= 200$
- $(A5 + B5) >= 700$

In [9]:
# Alternative 1: Writing out each constraint separately
# Supply maximum constraints
#        vars[w][b]
model += (vars["A"]["1"] + vars["A"]["2"] + vars["A"]["3"] + vars["A"]["4"] + vars["A"]["5"] <= supply["A"], 
          "Warehose_A_Supply_Maximum")
model += (vars["B"]["1"] + vars["B"]["2"] + vars["B"]["3"] + vars["B"]["4"] + vars["B"]["5"] <= supply["B"], 
          "Warehose_B_Supply_Maximum")

# Demand minimum constraints
model += (vars["A"]["1"] + vars["B"]["1"] >= demand["1"], "Bar_1_Demand_Minimum")
model += (vars["A"]["2"] + vars["B"]["2"] >= demand["2"], "Bar_2_Demand_Minimum")
model += (vars["A"]["3"] + vars["B"]["3"] >= demand["3"], "Bar_3_Demand_Minimum")
model += (vars["A"]["4"] + vars["B"]["4"] >= demand["4"], "Bar_4_Demand_Minimum")
model += (vars["A"]["5"] + vars["B"]["5"] >= demand["5"], "Bar_5_Demand_Minimum")


# # Alternative 2: Writing constraints using for loop and list comprehension
# # Supply maximum constraints
# for w in Warehouses:
#     model += (
#         lpSum([vars[w][b] for b in Bars]) <= supply[w],
#         f"Sum_of_Products_out_of_Warehouse_{w}",
#     )

# # Demand minimum constraints
# for b in Bars:
#     model += (
#         lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
#         f"Sum_of_Products_into_Bar{b}",
#     )
model

BeerDistribution:
MINIMIZE
2*Route_A_1 + 4*Route_A_2 + 5*Route_A_3 + 2*Route_A_4 + 1*Route_A_5 + 3*Route_B_1 + 1*Route_B_2 + 3*Route_B_3 + 2*Route_B_4 + 3*Route_B_5 + 0
SUBJECT TO
Warehose_A_Supply_Maximum: Route_A_1 + Route_A_2 + Route_A_3 + Route_A_4
 + Route_A_5 <= 1000

Warehose_B_Supply_Maximum: Route_B_1 + Route_B_2 + Route_B_3 + Route_B_4
 + Route_B_5 <= 4000

Bar_1_Demand_Minimum: Route_A_1 + Route_B_1 >= 500

Bar_2_Demand_Minimum: Route_A_2 + Route_B_2 >= 900

Bar_3_Demand_Minimum: Route_A_3 + Route_B_3 >= 1800

Bar_4_Demand_Minimum: Route_A_4 + Route_B_4 >= 200

Bar_5_Demand_Minimum: Route_A_5 + Route_B_5 >= 700

VARIABLES
0 <= Route_A_1 Integer
0 <= Route_A_2 Integer
0 <= Route_A_3 Integer
0 <= Route_A_4 Integer
0 <= Route_A_5 Integer
0 <= Route_B_1 Integer
0 <= Route_B_2 Integer
0 <= Route_B_3 Integer
0 <= Route_B_4 Integer
0 <= Route_B_5 Integer

In [10]:
# The problem data is written to an .lp file
model.writeLP("BeerDistribution.lp")

[Route_A_1,
 Route_A_2,
 Route_A_3,
 Route_A_4,
 Route_A_5,
 Route_B_1,
 Route_B_2,
 Route_B_3,
 Route_B_4,
 Route_B_5]

In [11]:
# Solve the optimization problem
status = model.solve()
status

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()} $ in total transportation costs")
for var in model.variables():
    print(f"{var.name}: {var.value()} crates")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()} crates")

status: 1, Optimal
objective: 8600.0 $ in total transportation costs
Route_A_1: 300.0 crates
Route_A_2: 0.0 crates
Route_A_3: 0.0 crates
Route_A_4: 0.0 crates
Route_A_5: 700.0 crates
Route_B_1: 200.0 crates
Route_B_2: 900.0 crates
Route_B_3: 1800.0 crates
Route_B_4: 200.0 crates
Route_B_5: 0.0 crates
Warehose_A_Supply_Maximum: 0.0 crates
Warehose_B_Supply_Maximum: -900.0 crates
Bar_1_Demand_Minimum: 0.0 crates
Bar_2_Demand_Minimum: 0.0 crates
Bar_3_Demand_Minimum: 0.0 crates
Bar_4_Demand_Minimum: 0.0 crates
Bar_5_Demand_Minimum: 0.0 crates


Total avaiable supply: 1000 + 4000 = 5000 crates   
Total minimum demand: 500 + 900 + 1800 + 200 + 700 = 4100 crates

Warehouse A supply 1000/1000 crates   
Warehouse B suppliess 3100/4000 crates

- ```Warehose_A_Supply_Maximum: 0.0 crates``` => Warehouse A exactly met its requirement by supplying 1000/1000 available crates.
- ```Warehose_B_Supply_Maximum: -900.0 crates``` => Warehouse B met its requirement without exceeding its max supply limit.  
It supplied 3100/4000 available crates.   
- ```Bar_1_Demand_Minimum: 0.0 crates```=> Bar 1 exactly met its requirement by receiving 500/500 crates.   
- ```Bar_2_Demand_Minimum: 0.0 crates```=> Bar 2 exactly met its requirement by receiving 900/900 crates.   
- ```Bar_3_Demand_Minimum: 0.0 crates```=> Bar 3 exactly met its requirement by receiving 1800/1800 crates.   
- ```Bar_4_Demand_Minimum: 0.0 crates```=> Bar 4 exactly met its requirement by receiving 200/200 crates.   
- ```Bar_5_Demand_Minimum: 0.0 crates```=> Bar 5 exactly met its requirement by receiving 700/700 crates. 

## Extensions to the Problem

Above, we solved an unbalanced model: The supply was 5000 crates, while the demand was only 4100 crates. No penalty for oversupply of crates.

As an extension to the problem, we can introduce 2 extensions:
- a) Introduce a cost of storing excess crates at the warehouses, when supply is greater than demand
- b) Introduce a cost of buying in crates from an external competitor, when demand is greater than supply

### a) Introducing Storage Costs
When supply is greater than demand, we can introduce a cost of storing the excess crates at the warehouses.  

This is added into the above model very simply. Since the objective function and constraints all operated on the original supply, demand and cost lists/dictionaries, the only changes that must be made is to include another demand node that holds the excess crates.

***For example, if we introduce a storage cost of 0.5 dollars per crate per week, then the objective function becomes:*** (remove???)

<img src="https://coin-or.github.io/pulp/_images/extra_supply.jpg" height="300"/>   


The Bars list is expanded and the Demand dictionary is expanded to make the Dummy Demand require 900 crates, to balance the problem. The cost list is also expanded, to show the cost of “sending to the Dummy Node” which is realistically just leaving the stock at the warehouses. This may have an associated cost which could be entered here instead of the zeros. Note that the solution could still be solved when there was an unbalanced excess supply.

In [28]:
# Creates a list of all the supply nodes
Warehouses = ["A", "B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000, "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5", "D"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 500, "2": 900, "3": 1800, "4": 200, "5": 700, "D": 900}

# Creates a list of costs of each transportation path
costs = [  # Bars
    # 1 2 3 4 5 D        # Add cost of sending crates to extra demand node D
    [2, 4, 5, 2, 1, 0],  # A   Warehouse
    [3, 1, 3, 2, 3, 0],  # B   Warehouse
]

# The cost data is made into a dictionary
costs = makeDict([Warehouses, Bars], costs, 0)

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Beer Distribution Problem", LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in Warehouses for b in Bars]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)

# The objective function is added to 'prob' first
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w],
        f"Sum_of_Products_out_of_Warehouse_{w}",
    )

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        f"Sum_of_Products_into_Bar{b}",
    )

prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_A_D = 0.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Route_B_D = 900.0
Total Cost of Transportation =  8600.0


If we increase the Storage cost, we can see that the value of the objective function (transportation cost) increase:
- 0 $ => The Total Cost of Transportation =  8600.0 \$ (same as unbalanced model)
- 0.5 $ => The Total Cost of Transportation =  9050.0 \$
- 1 $ => The Total Cost of Transportation =  9500.0 \$

### b) Introducing Buying From Competitor Costs

Note that with excess demand, the problem is “Infeasible” when unbalanced. This is because the model cannot be solved without violating the demand constraints.  
To balance the problem, we can introduce a Dummy Supply node that will provide the lacking crates to meet the demand requirements.   
The cost of buying in crates from an external competitor can be added to the objective function.

<img src="https://coin-or.github.io/pulp/_images/extra_demand.jpg" height="300"/>   

This dummy supply node 'C' is added in simply and logically into the Warehouse list, Supply dictionary, and costs list. The Supply value is chosen to balance the problem, and cost of transport is zero to all demand nodes.

In [35]:
# Creates a list of all the supply nodes
Warehouses = ["A", "B", "C"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000, "B": 3000, "C": 100}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 500, "2": 900, "3": 1800, "4": 200, "5": 700,}

# Creates a list of costs of each transportation path
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
    [10, 10, 10, 10, 10], # Cost of buying crates from competitor
]

# The cost data is made into a dictionary
costs = makeDict([Warehouses, Bars], costs, 0)

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Beer Distribution Problem", LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in Warehouses for b in Bars]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)

# The objective function is added to 'prob' first
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w],
        f"Sum_of_Products_out_of_Warehouse_{w}",
    )

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        f"Sum_of_Products_into_Bar{b}",
    )

# The problem data is written to an .lp file
prob.writeLP("BeerDistributionProblem.lp")

prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of Transportation = ", value(prob.objective))


Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 100.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Route_C_1 = 100.0
Route_C_2 = 0.0
Route_C_3 = 0.0
Route_C_4 = 0.0
Route_C_5 = 0.0
Total Cost of Transportation =  9300.0


If we increase the cost of buying in crates from an external competitor, we can see that the value of the objective function   
(transportation cost - crates from competitor) increase:
- 0 $ => The Total Cost of Transportation =  8300.0 \$
- 0.5 $ => The Total Cost of Transportation =  8350.0 \$
- 10 $ => The Total Cost of Transportation =  9300.0 \$